# Toka_LITE: 0D Tokamak Design Scan

Parametric study of a small tokamak using the Toka_LITE 0D model.
The scan covers aspect ratio and toroidal field and reports key plasma and coil quantities.

## 1. Add source path

Add the `src/` directory of Toka_LITE to the Python path.

In [ ]:
import sys, os

repo_root = os.path.abspath("../")
src_path = os.path.join(repo_root, "src")

if src_path not in sys.path:
    sys.path.append(src_path)

print("Repository root:", repo_root)
print("Using src path:", src_path)

## 2. Imports

Import Toka_LITE subroutines and numerical libraries.

In [ ]:
from subroutines.geometry_plasma import Shape, Ip_from_q95_PROCESS, greenwald_density, thermal_beta
from subroutines.tf_coils import TFCoils
from subroutines.scanning import scan_A_Bt_grid
from subroutines.plotting import plot_surface

import numpy as np
import matplotlib.pyplot as plt

## 3. Baseline parameters

Geometry, safety factor, density fraction, temperatures, and TF coil configuration for a university-scale tokamak.

In [ ]:
a = 0.25       # minor radius [m]
kappa = 1.6    # elongation
delta = 0.0    # triangularity
q95 = 4.0
f_G = 0.4      # Greenwald fraction
Te_eV = 100.0
Ti_eV = 50.0

tf = TFCoils(N_tf=12, N_turn=4, I_max_coil=15000.0)

print("Baseline Toka_LITE configuration initialised.")

## 4. (A, Bt) scan

Scan over aspect ratio and toroidal field for the chosen configuration.

In [ ]:
grids = scan_A_Bt_grid(
    A_min=1.5,
    A_max=3.0,
    Bt_min=0.25,
    Bt_max=0.75,
    n_A=60,
    n_Bt=60,
    a=a,
    kappa=kappa,
    delta=delta,
    q95=q95,
    f_G=f_G,
    Te_eV=Te_eV,
    Ti_e_V=Ti_eV,
    tf=tf,
)

print("Parameter scan completed.")

## 5. Extract fields

Convert to convenient units and apply feasibility mask from TF coil limit.

In [ ]:
A = grids["A"]
Bt = grids["Bt"]
Ip_kA = grids["Ip_MA"] * 1e3
ne = grids["ne"]
Icoil_kA = grids["Icoil"] * 1e-3
beta = grids["beta"]
mask = grids["feasible"]

Ip_kA = np.where(mask, Ip_kA, np.nan)
ne = np.where(mask, ne, np.nan)
Icoil_kA = np.where(mask, Icoil_kA, np.nan)
beta = np.where(mask, beta, np.nan)

print("Arrays prepared for plotting.")

## 6. Surface plots

3D surfaces for plasma current, density, TF coil current, and thermal beta as functions of aspect ratio and toroidal field.

In [ ]:
plot_surface(A, Bt, Ip_kA, "A", "Bt [T]", "Ip [kA]", "Ip vs (A, Bt)")
plot_surface(A, Bt, ne, "A", "Bt [T]", "ne [m^-3]", "ne vs (A, Bt)")
plot_surface(A, Bt, Icoil_kA, "A", "Bt [T]", "I_coil [kA]", "TF coil current vs (A, Bt)")
plot_surface(A, Bt, beta, "A", "Bt [T]", "beta", "beta vs (A, Bt)")

plt.show()

## 7. Summary

The Toka_LITE 0D scan provides a compact view of how basic geometric and magnetic parameters impact plasma current, density, coil loading and beta for a university-scale tokamak.